In [ ]:
%%capture
!pip install gdown

In [ ]:
import gdown

import pandas as pd
import matplotlib.pyplot as plt

import xml.etree.ElementTree as ET

from pandas.plotting import autocorrelation_plot

In [ ]:
%%capture

ROUTE_GPX = 'route.gpx'
ROUTE_GPX_URL = 'https://drive.google.com/file/d/1ypx5oiromH9WRQpqAPNxZYQO6T3Dd0fP/view?usp=sharing'

gdown.download(ROUTE_GPX_URL, ROUTE_GPX, quiet=True, fuzzy=True)

In [ ]:
def parse_gpx(filepath):
    root = ET.parse(filepath).getroot()

    ns = '{http://www.topografix.com/GPX/1/1}'
    data = {
        'time': [], 'lat': [], 'lon': [],
    }

    for trk in root.findall(f".//{ns}trk"):
        for trkseg in trk.findall(f"{ns}trkseg"):
            for trkpt in trkseg.findall(f"{ns}trkpt"):
                data['lat'].append(float(trkpt.get('lat')))
                data['lon'].append(float(trkpt.get('lon')))

                time_elem = trkpt.find(f"{ns}time")
                data['time'].append(time_elem.text if time_elem is not None else None)

    df = pd.DataFrame(data)

    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)

    return df

df = parse_gpx(ROUTE_GPX)
df.head()

In [ ]:
plt.title('Lat distbution')
df['lat'].plot(kind='hist', bins=30, density=True, alpha=0.5)
df['lat'].plot(kind='kde', color='r')

In [ ]:
plt.title('Lon distbution')
df['lon'].plot(kind='hist', bins=30, density=True, alpha=0.5)
df['lon'].plot(kind='kde', color='r')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['lat'])
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['lon'])
plt.show()

In [ ]:
autocorrelation_plot(df['lat'])
plt.show()

In [ ]:
autocorrelation_plot(df['lon'])
plt.show()